In [1]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.recurrent import SimpleRNN
from keras.layers.embeddings import Embedding
from keras.layers import LSTM
import numpy as np

/root/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# define documents
docs = ['very good',
		'very bad']
# define class labels
labels = [1,0]
from collections import Counter
counts = Counter()
for i,review in enumerate(docs):
    counts.update(review.split())

In [3]:
words = sorted(counts, key=counts.get, reverse=True)
vocab_size=len(words)

In [4]:
word_to_int = {word: i for i, word in enumerate(words, 1)}

In [5]:
encoded_docs = []
for doc in docs:
    encoded_docs.append([word_to_int[word] for word in doc.split()])

In [6]:
encoded_docs

[[1, 3], [1, 2]]

In [7]:
# pad documents to a max length of 2 words
max_length = 2
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='pre')
print(padded_docs)

[[1 3]
 [1 2]]


In [8]:
model = Sequential()
model.add(SimpleRNN(1,activation='tanh', return_sequences=False,input_shape=(2,1),unroll=True))
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 1)                 3         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
# define the model
embed_length=1
model = Sequential()
#model.add(Embedding(vocab_size, embed_length, input_length=max_length))
model.add(SimpleRNN(1,activation='tanh', return_sequences=False,recurrent_initializer='Zeros',input_shape=(max_length,embed_length),unroll=True))
#model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
#model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 1)                 3         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 2         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
model.fit(padded_docs.reshape(2,2,1),np.array(labels).reshape(max_length,1),epochs=500)

Epoch 1/500
2/2 [==============================] - 0s 245ms/step - loss: 0.7276 - acc: 0.5000
Epoch 2/500
2/2 [==============================] - 0s 805us/step - loss: 0.7273 - acc: 0.5000
Epoch 3/500
2/2 [==============================] - 0s 579us/step - loss: 0.7270 - acc: 0.5000
Epoch 4/500
2/2 [==============================] - 0s 738us/step - loss: 0.7268 - acc: 0.5000
Epoch 5/500
2/2 [==============================] - 0s 647us/step - loss: 0.7265 - acc: 0.5000
Epoch 6/500
2/2 [==============================] - 0s 644us/step - loss: 0.7262 - acc: 0.5000
Epoch 7/500
2/2 [==============================] - 0s 566us/step - loss: 0.7259 - acc: 0.5000
Epoch 8/500
2/2 [==============================] - 0s 656us/step - loss: 0.7257 - acc: 0.5000
Epoch 9/500
2/2 [==============================] - 0s 655us/step - loss: 0.7254 - acc: 0.5000
Epoch 10/500
2/2 [==============================] - 0s 682us/step - loss: 0.7251 - acc: 0.5000
Epoch 11/500
2/2 [==============================] - 0s 650u

In [11]:
padded_docs

array([[1, 3],
       [1, 2]], dtype=int32)

In [12]:
model.layers

In [13]:
model.weights

[<tf.Variable 'simple_rnn_2/kernel:0' shape=(1, 1) dtype=float32_ref>,
 <tf.Variable 'simple_rnn_2/recurrent_kernel:0' shape=(1, 1) dtype=float32_ref>,
 <tf.Variable 'simple_rnn_2/bias:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'dense_2/kernel:0' shape=(1, 1) dtype=float32_ref>,
 <tf.Variable 'dense_2/bias:0' shape=(1,) dtype=float32_ref>]

In [14]:
model.get_weights()

[array([[0.56373304]], dtype=float32),
 array([[-0.50989217]], dtype=float32),
 array([-0.69803804], dtype=float32),
 array([[0.5092683]], dtype=float32),
 array([-0.27220774], dtype=float32)]

In [15]:
model.layers[0].get_weights()

[array([[0.56373304]], dtype=float32),
 array([[-0.50989217]], dtype=float32),
 array([-0.69803804], dtype=float32)]

In [16]:
padded_docs[0].reshape(1,2,1)

array([[[1],
        [3]]], dtype=int32)

In [17]:
import numpy as np
model.predict(padded_docs[0].reshape(1,2,1))

array([[0.53199273]], dtype=float32)

In [18]:
names = [weight.name for layer in model.layers for weight in layer.weights]
weights = model.get_weights()

for name, weight in zip(names, weights):
    print(name, weight.shape)

simple_rnn_2/kernel:0 (1, 1)
simple_rnn_2/recurrent_kernel:0 (1, 1)
simple_rnn_2/bias:0 (1,)
dense_2/kernel:0 (1, 1)
dense_2/bias:0 (1,)


In [19]:
model = Sequential()
model.add(LSTM(1,activation='tanh', return_sequences=False,recurrent_initializer='Zeros',recurrent_activation='sigmoid',
               input_shape=(2,1),unroll=True))
model.add(Dense(1, activation='sigmoid'))
#model.add(Dense(1))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1)                 12        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 2         
Total params: 14
Trainable params: 14
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
model.fit(padded_docs.reshape(2,2,1),np.array(labels).reshape(max_length,1),epochs=500)

Epoch 1/500
2/2 [==============================] - 0s 199ms/step - loss: 0.7267 - acc: 0.5000
Epoch 2/500
2/2 [==============================] - 0s 880us/step - loss: 0.7264 - acc: 0.5000
Epoch 3/500
2/2 [==============================] - 0s 763us/step - loss: 0.7261 - acc: 0.5000
Epoch 4/500
2/2 [==============================] - 0s 706us/step - loss: 0.7257 - acc: 0.5000
Epoch 5/500
2/2 [==============================] - 0s 739us/step - loss: 0.7254 - acc: 0.5000
Epoch 6/500
2/2 [==============================] - 0s 667us/step - loss: 0.7250 - acc: 0.5000
Epoch 7/500
2/2 [==============================] - 0s 778us/step - loss: 0.7247 - acc: 0.5000
Epoch 8/500
2/2 [==============================] - 0s 714us/step - loss: 0.7244 - acc: 0.5000
Epoch 9/500
2/2 [==============================] - 0s 688us/step - loss: 0.7241 - acc: 0.5000
Epoch 10/500
2/2 [==============================] - 0s 759us/step - loss: 0.7237 - acc: 0.5000
Epoch 11/500
2/2 [==============================] - 0s 725u

In [110]:
model.layers

In [111]:
model.get_config()

[{'class_name': 'LSTM',
  'config': {'activation': 'tanh',
   'activity_regularizer': None,
   'batch_input_shape': (None, 2, 1),
   'bias_constraint': None,
   'bias_initializer': {'class_name': 'Zeros', 'config': {}},
   'bias_regularizer': None,
   'dropout': 0.0,
   'dtype': 'float32',
   'go_backwards': False,
   'implementation': 1,
   'kernel_constraint': None,
   'kernel_initializer': {'class_name': 'VarianceScaling',
    'config': {'distribution': 'uniform',
     'mode': 'fan_avg',
     'scale': 1.0,
     'seed': None}},
   'kernel_regularizer': None,
   'name': 'lstm_10',
   'recurrent_activation': 'sigmoid',
   'recurrent_constraint': None,
   'recurrent_dropout': 0.0,
   'recurrent_initializer': {'class_name': 'Zeros', 'config': {}},
   'recurrent_regularizer': None,
   'return_sequences': False,
   'return_state': False,
   'stateful': False,
   'trainable': True,
   'unit_forget_bias': True,
   'units': 1,
   'unroll': True,
   'use_bias': True}},
 {'class_name': 'Dense',

In [112]:
model.layers[0].get_weights()

[array([[-0.32939184, -0.9811073 , -0.6059296 , -0.33957732]],
       dtype=float32),
 array([[-0.04132899, -0.71698403,  0.55261445, -0.37336   ]],
       dtype=float32),
 array([ 0.24954776,  1.6016548 , -0.4455776 ,  0.26094177], dtype=float32)]

In [113]:
model.layers[0].trainable_weights

[<tf.Variable 'lstm_10/kernel:0' shape=(1, 4) dtype=float32_ref>,
 <tf.Variable 'lstm_10/recurrent_kernel:0' shape=(1, 4) dtype=float32_ref>,
 <tf.Variable 'lstm_10/bias:0' shape=(4,) dtype=float32_ref>]

In [114]:
model.layers[1].get_weights()

[array([[1.7236937]], dtype=float32), array([0.24704114], dtype=float32)]

In [118]:
model.layers[1].trainable_weights

[<tf.Variable 'dense_10/kernel:0' shape=(1, 1) dtype=float32_ref>,
 <tf.Variable 'dense_10/bias:0' shape=(1,) dtype=float32_ref>]

In [115]:
padded_docs[1].reshape(1,2,1)

array([[[1],
        [2]]], dtype=int32)

In [117]:
model.predict(padded_docs[1].reshape(1,2,1))

array([[0.47375038]], dtype=float32)

In [158]:
model.layers

In [37]:
model.layers[0].get_weights()

[array([[ 0.03574599,  0.03463735,  0.03284376,  0.02637372,  0.03980382,
         -0.03907117, -0.02789926, -0.00358451],
        [ 0.03421954,  0.04351522,  0.00589408, -0.04832727, -0.00828198,
         -0.04772505,  0.00875548, -0.01536892],
        [ 0.04898155,  0.01730723, -0.02951646,  0.02846657,  0.01706989,
          0.03087733, -0.00581173, -0.03555614],
        [ 0.03407046,  0.03600022, -0.03234861,  0.03110384, -0.0099007 ,
          0.01107918,  0.00615746, -0.03383043],
        [ 0.00602891,  0.00023402,  0.02693779, -0.02934064, -0.04537648,
         -0.0072022 , -0.02867633,  0.00901375],
        [-0.01281931,  0.01679485,  0.04794529,  0.03495849, -0.04389096,
          0.00275611,  0.02929867,  0.01125343],
        [ 0.00855082, -0.03869943,  0.00248744, -0.00599202,  0.03802994,
          0.04899568, -0.04703009, -0.0290049 ],
        [ 0.03235635, -0.01593938, -0.0316224 , -0.047445  , -0.01225562,
         -0.02097278, -0.01437372, -0.00120188],
        [ 0.0277